In [1]:
from pulp import *
from pulp import LpProblem, LpVariable, LpMinimize, LpInteger, lpSum, value, LpBinary,LpStatusOptimal
import pulp
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore", message="Overwriting previously set objective.")
import utility
import docplex.mp.model
import docplex
import docplex_explainer
import mymetrics

In [2]:
# Load Dataset
dataset = datasets.load_iris()
dataset_name = 'Iris'
df = pd.DataFrame(dataset.data, columns = dataset.feature_names)

In [3]:
# Scale
scaler = MinMaxScaler()
scaler.fit(dataset.data)
scaled_df = scaler.transform(dataset.data)

In [4]:
# Get scaled bounds
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)

0.0 1.0


In [5]:
# Check if binary targets
df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
df_scaled['target'] = targets

Original Targets:  [0 1] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True


In [6]:
# Train model
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.3,random_state=50,stratify=targets)
X = np.concatenate((X_train,X_test),axis=0)
y = np.concatenate((y_train,y_test),axis=0)

clf = svm.SVC(kernel='linear')

# Train the model using the training set
clf.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = clf.predict(X_test)
print("Accuracy Linear:", metrics.accuracy_score(y_test, y_pred))

Accuracy Linear: 1.0


In [7]:
# Finding patterns classified as positive/negative
positive_indexes,negative_indexes = utility.find_indexes(clf, X_test, threshold=0)
print(f"Positive patterns = {len(positive_indexes)},\nNegative patterns = {len(negative_indexes)}")

Positive patterns = 30,
Negative patterns = 15


In [8]:
# Make a dataframe with the test data. For comparing Onestep against Twostep.
test_df_names = list(df.columns)
if 'target' not in test_df_names:
    test_df_names.append('target')
test_dataset = []
for instance, test_class in zip(X_test, y_test.astype('int32')):
    test_dataset.append(np.append(instance, test_class))
test_dataset_df = pd.DataFrame(np.asarray(test_dataset), columns=test_df_names)

In [9]:
# Parameter p value
p = 0.5

In [10]:
# Instance to be explained
instance = X_test[negative_indexes[0:1]]
instance

array([[0.30555556, 0.79166667, 0.11864407, 0.125     ]])

In [11]:
scaler.inverse_transform(np.atleast_2d(instance))

array([[5.4, 3.9, 1.7, 0.4]])

In [12]:
clf.predict(instance)

array([0])

In [13]:
#Onestep
onestep_exp = docplex_explainer.onestep(
        classifier = clf,
        dual_coef = clf.dual_coef_,
        support_vectors = clf.support_vectors_,
        intercept = clf.intercept_,
        lower_bound = lower_bound,
        upper_bound = upper_bound,
        data = (instance),
        positive = False)

In [14]:
onestep_exp

[[0.0, 1.0],
 [0.352021508800505, 1.0],
 [0.0, 0.11864406779661013],
 [0.0, 0.125]]

In [15]:
#Onestep Lowerbound
scaler.inverse_transform(np.atleast_2d([0,
                                       0.352021508800505,
                                       0,
                                       0]))

array([[4.3       , 2.84485162, 1.        , 0.1       ]])

In [16]:
#Onestep Upperbound
scaler.inverse_transform(np.atleast_2d([1.0,
                                       1.0,
                                       0.11864406779661013,
                                       0.125]))

array([[7.9, 4.4, 1.7, 0.4]])

In [17]:
#Twostep
twostep_exp = docplex_explainer.twostep(
        classifier = clf,
        dual_coef = clf.dual_coef_,
        support_vectors = clf.support_vectors_,
        intercept = clf.intercept_,
        lower_bound = lower_bound,
        upper_bound = upper_bound,
        data = (instance),
        p = p,
        positive = False)

In [18]:
twostep_exp

[[0.0, 1.0],
 [0.5137114203688846, 1.0],
 [0.0, 0.18765790040017305],
 [0.0, 0.16512991270501043]]

In [19]:
#Twostep Lowerbound
scaler.inverse_transform(np.atleast_2d([0,
                                       0.5137114203688846,
                                       0,
                                       0]))

array([[4.3       , 3.23290741, 1.        , 0.1       ]])

In [20]:
#Twostep Powerbound
scaler.inverse_transform(np.atleast_2d([1,
                                       1,
                                       0.18765790040017305,
                                       0.16512991270501043]))

array([[7.9       , 4.4       , 2.10718161, 0.49631179]])